<a href="https://colab.research.google.com/github/KOO-96/DNA_seminar/blob/main/DNA_ComputerVisionSeminar_Weeks2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Weeks2
1. Transfer Learning (전이 학습)
   - 다른 이의 가중치를 그대로 이용하려함.
   - Feature Extraction
- Imagenet Chanllenge
- Transfer Learning

- Fine Tuning: 미세 조정.
- Backbone
  - Vggnet 사용했으면 Vggnet의 CNN단
  - ResNet 사용했으면 ResNet의 CNN단

## Weeks2 과제 solution.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 현재 경로를 앞으로 진행할 프로젝트 디렉토리로 이동한다.
%cd /content/drive/MyDrive/CV_seminar_project 

/content/drive/MyDrive/CV_seminar_project


In [ ]:
# 현재 경로 확인.
%pwd

'/content/drive/MyDrive/CV_seminar_project'

In [ ]:
import glob

dolphin = glob.glob('/content/drive/MyDrive/CV_seminar_project/original/dolphin/*.jpg')
shark = glob.glob('/content/drive/MyDrive/CV_seminar_project/original/shark/*.jpg')
whale = glob.glob('/content/drive/MyDrive/CV_seminar_project/original/whale/*.jpg')

dic = {'dolphin':dolphin, 'shark': shark, 'whale': whale}
for key in dic.keys():
  print(f'{key}이미지가 ',len(dic[key]), '개 있습니다.')
print('------------------------------------------------------------------------')

length_list = []
for key in dic.keys():
  print(f'{key}이미지는 trian, valid, test셋에 대해 ',int(len(dic[key])*0.7), int(len(dic[key])*0.2), int(len(dic[key])*0.1), '개씩 배정해주세요.')
  length_list.append([int(len(dic[key])*0.7), int(len(dic[key])*0.2), int(len(dic[key])*0.1)])

dolphin이미지가  373 개 있습니다.
shark이미지가  488 개 있습니다.
whale이미지가  451 개 있습니다.
------------------------------------------------------------------------
dolphin이미지는 trian, valid, test셋에 대해  261 74 37 개씩 배정해주세요.
shark이미지는 trian, valid, test셋에 대해  341 97 48 개씩 배정해주세요.
whale이미지는 trian, valid, test셋에 대해  315 90 45 개씩 배정해주세요.


## 직접 짠 코드.

In [ ]:
import os, shutil, random
import glob
import cv2
import time

class Make_dataset_dir():
  def __init__(self, root_dir):
    self.root_path = root_dir+'/' if root_dir[-1] != '/' else root_dir 
    # 현재 진행할 프로젝트 -> root path는 /content/drive/MyDrive/CV_seminar_project/ 가 되어야합니다.
    self.img_path_list = root_dir+'original' # 전달한 이미지들의 상위 경로
    self.trainset_path = root_dir+'train/'
    self.validset_path = root_dir+'valid/'
    self.testset_path = root_dir+'test/'
    self.class_list = ['dolphin', 'shark', 'whale']

  def mk_dir(self):
    '''train, valid, test 폴더를 만들고, 내부에는 클래스 별 폴더를 추가로 만들어 주세요.'''
    for i in self.class_list:
      train = self.trainset_path + i
      valid = self.validset_path + i
      test = self.testset_path + i
      for path in [train, valid, test]:
        os.makedirs(path, exist_ok = True)
    
  def move_img(self):
    # dolphin path
    train_dol_path = self.trainset_path + 'dolphin'
    valid_dol_path = self.validset_path + 'dolphin'
    test_dol_path = self.testset_path + 'dolphin'
    # shark path
    train_sh_path = self.trainset_path + 'shark'
    valid_sh_path = self.validset_path + 'shark'
    test_sh_path = self.testset_path + 'shark'
    # whale_path
    train_wh_path = self.trainset_path + 'whale'
    valid_wh_path = self.validset_path + 'whale'
    test_wh_path = self.testset_path + 'whale'

    # original 폴더 내 파일명 리스트로 반환
    dol_path = self.img_path_list +'/dolphin'
    sh_path = self.img_path_list +'/shark'
    wh_path = self.img_path_list +'/whale'

    dolphin_list = os.listdir(dol_path)
    shark_list = os.listdir(sh_path)
    whale_list = os.listdir(wh_path)

    #dolphin 7:3:1 분류 진행 -> 차집합 이용  train: a1 ; valid: b1 ; test:c1
    a1, b1, c1 = [], [], []
    for j in random.sample(dolphin_list, round(len(dolphin)*0.70)):
      a1.append(j)
      b0 = [x for x in dolphin_list if x not in a1]
    for k in random.sample(b0, round(len(b0)*0.67)):
      b1.append(k)
      c1 = [x for x in b0 if x not in b1]


    #shark 7:3:1 분류 진행 -> 차집합 이용  train: a2 ; valid: b2 ; test:c2
    a2, b2, c2 = [], [], []
    for l in random.sample(shark_list, round(len(shark)*0.70)):
      a2.append(l)
      b4 = [x for x in shark_list if x not in a2]
    for m in random.sample(b4, round(len(b4)*0.67)):
      b2.append(m)
      c2 = [x for x in b4 if x not in b2]


    #whale 7:3:1 분류 진행 -> 차집합 이용  train: a3 ; valid: b3 ; test:c3
    a3, b3, c3 = [], [], []
    for n in random.sample(whale_list, round(len(whale)*0.70)):
      a3.append(n)
      b5 = [x for x in whale_list if x not in a3]
    for k in random.sample(b5, round(len(b5)*0.67)):
      b3.append(k)
      c3 = [x for x in b5 if x not in b3]

    # 폴더 이동 original -> train, valid, test
    for i in dolphin_list:
      if i in a1:
        shutil.copy(dol_path + '/' + i, train_dol_path + '/' + i)
      elif i in b1:
        shutil.copy(dol_path + '/' + i, valid_dol_path + '/' + i)
      elif i in c1:
        shutil.copy(dol_path + '/' + i, test_dol_path + '/' + i)

    for i in shark_list:
      if i in a2:
        shutil.copy(sh_path + '/' + i, train_sh_path + '/' + i)
      elif i in b2:
        shutil.copy(sh_path + '/' + i, valid_sh_path + '/' + i)
      elif i in c2:
        shutil.copy(sh_path + '/' + i, test_sh_path + '/' + i)

    for i in whale_list:
      if i in a3:
        shutil.copy(wh_path + '/' + i, train_wh_path + '/' + i)
      elif i in b3:
        shutil.copy(wh_path + '/' + i, valid_wh_path + '/' + i)
      elif i in c3:
        shutil.copy(wh_path + '/' + i, test_wh_path + '/' + i)

  def run(self):
    start = time.time()
    self.mk_dir()
    self.move_img()
    print('총 소요시간: ', time.time()-start)

  def checking_dirs(self):
    path_list = [self.trainset_path, self.validset_path, self.testset_path]
    for i,path in enumerate(path_list):
      length_dic = {}
      for cls in self.class_list:
        length_dic[cls] = len(glob.glob(path+cls+'/*'))

      if i==0:
        for key in length_dic:
          print( f'trainset의 {key}클래스 개수: {length_dic[key]}')
        print('---------------------------------------------------------------')
      elif i==1:
        for key in length_dic:
          print( f'validset의 {key}클래스 개수: {length_dic[key]}')
        print('---------------------------------------------------------------')
      else:
        for key in length_dic:
          print( f'validset의 {key}클래스 개수: {length_dic[key]}')


## Solution 코드.

In [ ]:
import os
import glob
import cv2
import time

class Make_dataset_dir():
  def __init__(self, root_dir):
    self.root_path = root_dir+'/' if root_dir[-1] != '/' else root_dir # 현재 진행할 프로젝트
    self.img_path_list = root_dir+'original' # 전달한 이미지들의 상위 경로
    self.trainset_path = root_dir+'train/'
    self.validset_path = root_dir+'valid/'
    self.testset_path = root_dir+'test/'
    self.class_list = ['dolphin', 'shark', 'whale']

  def mk_dir(self):
    '''train, valid, test 폴더를 만들고, 내부에는 클래스 별 폴더를 추가로 만들어 주세요.'''
    dataset_dir_list = [self.trainset_path, self.validset_path, self.testset_path]
    for dataset_dir in dataset_dir_list:
      for cls in self.class_list:
        os.makedirs(dataset_dir+cls, exist_ok=True)
    print('디렉토리 생성을 완료하였습니다.')
    
  def move_img(self):
    '''mk_dir에서 만든 폴더들에 각 클래스에 맞는 이미지를 배당해주세요. train, valid, test에 각각 7: 2: 1'''
    dolphin_img_list = glob.glob('/content/drive/MyDrive/CV_seminar_project/original/dolphin/*')
    shark_img_list = glob.glob('/content/drive/MyDrive/CV_seminar_project/original/shark/*')
    whale_img_list = glob.glob('/content/drive/MyDrive/CV_seminar_project/original/whale/*')
    
    dic = {'dolphin':dolphin_img_list, 'shark': shark_img_list, 'whale': whale_img_list} # 딕셔너리 생성
    for key in dic.keys():
      length_list.append([int(len(dic[key])*0.7), int(len(dic[key])*0.2), int(len(dic[key])*0.1)]) # 클래스 별 분할 개수 리스트 생성

    for i,key in enumerate(dic.keys()):
      spliting_length = length_list[i]

      for ii, img_path in enumerate(dic[key]):
        if ii+1 <= spliting_length[0] : # train셋 개수이면,
          img = cv2.imread(img_path)
          img_name = img_path.split('/')[-1]
          cv2.imwrite(self.trainset_path + '/' + key + '/' + img_name, img)

        elif spliting_length[0] < ii+1 and ii+1 <= spliting_length[0] + spliting_length[1]: # valid셋 개수이면,
          img = cv2.imread(img_path)
          img_name = img_path.split('/')[-1]
          cv2.imwrite(self.validset_path + '/' + key + '/' + img_name, img)

        else:
          img = cv2.imread(img_path)
          img_name = img_path.split('/')[-1]
          cv2.imwrite(self.testset_path + '/' + key + '/' + img_name, img)
    print('데이터 스플릿이 전부 완료되었습니다.')

  def run(self):
    start = time.time()
    self.mk_dir()
    self.move_img()
    print('총 소요시간: ', time.time()-start)

  def checking_dirs(self):
    path_list = [self.trainset_path, self.validset_path, self.testset_path]

    for i,path in enumerate(path_list):
      length_dic = {}
      for cls in self.class_list:
        length_dic[cls] = len(glob.glob(path+cls+'/*'))

      if i==0:
        for key in length_dic:
          print( f'trainset의 {key}클래스 개수: {length_dic[key]}')
        print('---------------------------------------------------------------')
      elif i==1:
        for key in length_dic:
          print( f'validset의 {key}클래스 개수: {length_dic[key]}')
        print('---------------------------------------------------------------')
      else:
        for key in length_dic:
          print( f'validset의 {key}클래스 개수: {length_dic[key]}')

-> 직접 짠 코드에 대한 의견: 가독성이 좋지못했음.